In [1]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, Bidirectional, Dropout, Dense, Flatten, Reshape
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.optimizers import Adam, SGD
import os
import json
import numpy as np
import random
from sklearn import metrics
import matplotlib.pyplot as plt
import time
from tensorflow.keras.callbacks import Callback, CSVLogger

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')


""" 
folder structure:

GDP
  test
  train
  weights

"""
# Check which directory are you in:

# your working directory should be in the main folder where "weights" folder is in.

# print(os.getcwd())

# %cd /content/drive/MyDrive/GDP 




Mounted at /content/drive
/content/drive/MyDrive/GDP


### Custom Callback

In [3]:
import time
from tensorflow.keras.callbacks import Callback, CSVLogger
def compute_test(model, X_test_reshaped, y_test):
    # Make predictions on the test data
    y_pred = model.predict(X_test_reshaped)

    # Convert the predicted probabilities to class labels
    y_pred_classes = y_pred.argmax(axis=-1)

    # Compute test accuracy
    correct=0
    for i in range(len(y_pred_classes)):
        if y_pred_classes[i]==y_test[i]:
            correct+=1
    test_accuracy=correct/len(y_pred_classes)

    return test_accuracy, y_pred_classes

class SaveWeightsJSON(Callback):
    def __init__(self, X_test_reshaped, y_test, param_idx, sup_hyper_param_list, model_type, num_frames, num_features, weights_path):
        super(SaveWeightsJSON, self).__init__()
        self.start_time = time.time()
        self.model_name=f"model{param_idx}"
        self.weights_path = weights_path
        self.X_test = X_test_reshaped
        self.y_test = y_test
        
        self.json_data={
                "model_name": self.model_name,
                "model_type": model_type,
                "num_frames": num_frames,
                "num_features": num_features, 
                "num_layers": sup_hyper_param_list[0],
                "dropout_rate": sup_hyper_param_list[1],
                "optimizer": sup_hyper_param_list[2],
                "batch_size": sup_hyper_param_list[3],
                "learning_rate": sup_hyper_param_list[4],
                "best_epoch": 0,
                "best_train_acc": 0.0,
                "best_test_acc": 0.0,
                "training_time": 0.0
        }

        self.best_epoch = 0
        self.best_val_loss = float('inf')
        self.best_train_accuracy = 0.0
        self.best_test_accuracy = 0.0

    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs.get('val_loss')
        test_acc, _ = compute_test(self.model, self.X_test, self.y_test)

        if val_loss <= self.best_val_loss:
            if test_acc > self.best_test_accuracy:
                self.best_val_loss = val_loss

                self.best_epoch = epoch + 1
                self.best_train_accuracy = logs.get('accuracy')
                self.best_test_accuracy = test_acc
                weights_file_path = os.path.join(self.weights_path, f"{self.model_name}.h5")
                self.model.save_weights(weights_file_path, overwrite=True)
                                
    def on_train_end(self, logs=None):
        self.json_data['best_epoch'] = self.best_epoch
        self.json_data['best_train_acc'] = self.best_train_accuracy
        self.json_data['best_test_acc'] = self.best_test_accuracy
        json_file_path = os.path.join(self.weights_path, f"{self.model_name}.json")
        self.json_data['training_time'] = time.time() - self.start_time
        with open(json_file_path, 'w') as f:
            json.dump(self.json_data, f, indent=2)


### Main Modeling Function (including reading data & preprocessing data & create and fit the desired model to the data)

In [4]:
# Imports

import os
import json
import numpy as np
import random
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, Bidirectional, Dropout, Dense, Flatten, Reshape
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD

from sklearn import metrics
import matplotlib.pyplot as plt

def load_data(split_path):
  X = []
  Y = []
  seqs_list=os.listdir(split_path)
  random.shuffle(seqs_list)
  for seq in seqs_list:
    seq_file_path=os.path.join(split_path,seq)
    with open(seq_file_path, "r") as f:
      data = json.load(f)
      X.append(data['features'])
      Y.append(data['class'])
  X = np.array(X)
  Y = np.array(Y)

  return X,Y

def opt_function(optimizer,learning_rate):

  if optimizer == 'adam':
    opt = Adam(learning_rate=learning_rate)
  else:
    opt = SGD(learning_rate=learning_rate)

  return opt

def compile_model(sup_hyper_param_list,lstm_units,model_type, num_frame, num_features):

  num_kpsORjnts=int(num_features/2)
  num_layer= sup_hyper_param_list[0]
  dropout_rate= sup_hyper_param_list[1]
  optimizer= sup_hyper_param_list[2]
  #batch_size= sup_hyper_param_list[3]
  learning_rate= sup_hyper_param_list[4]

  opt = opt_function(optimizer,learning_rate)

  num_layer_list = list(range(0,num_layer))
  num_layer_list.reverse()


# Define paramters used in sup_hyper_param_list

  if model_type == 'LSTM':

    model = Sequential ()
    model.add(LSTM(units = (lstm_units)*(2**num_layer), return_sequences=True, input_shape=(num_frame,num_features)))
    model.add(Dropout(dropout_rate))

    for s in num_layer_list:

      if s == 0:
        model.add(LSTM(units = (lstm_units)*(2**s), return_sequences=False))
      else:

        model.add(LSTM(units = (lstm_units)*(2**s), return_sequences=True))
      model.add(Dropout(dropout_rate))
    model.add(Dense(3, activation='softmax'))
    #model.summary()
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

  elif model_type == 'CNN_LSTM':
    

    model = Sequential ()
    model.add(Conv2D(16, kernel_size=(5, 5), activation='relu', input_shape=(num_frame,num_kpsORjnts,2)))
    model.add(Dropout(dropout_rate))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Reshape((model.output_shape[1],model.output_shape[2]*model.output_shape[3])))
    model.add(LSTM(units = (lstm_units)*(2**num_layer), return_sequences=True))
    model.add(Dropout(dropout_rate))

    for s in num_layer_list:
      
      if s == 0:
        model.add(LSTM(units = (lstm_units)*(2**s), return_sequences=False))
      else:

        model.add(LSTM(units = (lstm_units)*(2**s), return_sequences=True))
      model.add(Dropout(dropout_rate))
    model.add(Dense(3, activation='softmax'))
    #model.summary()
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

  elif model_type == 'BiLSTM':

    model = Sequential ()
    model.add(Bidirectional(LSTM(units = (lstm_units)*(2**num_layer),return_sequences=True), input_shape=(num_frame,num_features)))
    model.add(Dropout(dropout_rate))

    for s in num_layer_list:

      if s == 0:
        model.add(Bidirectional(LSTM(units = (lstm_units)*(2**s),return_sequences=False)))
      else:

        model.add(Bidirectional(LSTM(units = (lstm_units)*(2**s),return_sequences=True)))
      model.add(Dropout(dropout_rate))
    model.add(Dense(3, activation='softmax'))
    #model.summary()
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model
  
  elif model_type == 'CNN_BiLSTM':
  
    model = Sequential ()
    model.add(Conv2D(16, kernel_size=(5, 5), activation='relu', input_shape=(num_frame,num_kpsORjnts,2)))
    model.add(Dropout(dropout_rate))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Reshape((model.output_shape[1],model.output_shape[2]*model.output_shape[3])))
    model.add(Bidirectional(LSTM(units = (lstm_units)*(2**num_layer),return_sequences=True), input_shape=(num_frame,num_features)))
    model.add(Dropout(dropout_rate))
    for s in num_layer_list:

      if s == 0:
        model.add(Bidirectional(LSTM(units = (lstm_units)*(2**s),return_sequences=False)))
      else:

        model.add(Bidirectional(LSTM(units = (lstm_units)*(2**s),return_sequences=True)))
      model.add(Dropout(dropout_rate))
    model.add(Dense(3, activation='softmax'))
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model
  else:
    print('Invalid model name. You can only use these models: "CNN","CNN_LSTM","BiLSTM","CNN_BiLSTM"')


def run_models(hyper_param_list,epochs,lstm_units,split_train_path,split_test_path,weights_path,model_type):

    print("Number of configurations to run: ", len(hyper_param_list))
    X_train, y_train = load_data(split_train_path)
    num_json,num_frames,num_features = X_train.shape 

    X_test, y_test = load_data(split_test_path)
    num_json_test = X_test.shape[0]

    num_classes = 3
    y_train_cat=to_categorical(y_train,num_classes)
    y_test_cat=to_categorical(y_test,num_classes)

    num_kpsORjnts=int(num_features/2)

    if model_type == 'CNN_LSTM' or model_type == 'CNN_BiLSTM':

      X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], num_kpsORjnts, 2))
      X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], num_kpsORjnts, 2))

    else:
      pass

    for param_idx, sup_hyper_param_list in enumerate(hyper_param_list): 
      print(sup_hyper_param_list)
      save_weights_json = SaveWeightsJSON(X_test_reshaped=X_test, y_test=y_test, param_idx=param_idx,sup_hyper_param_list=sup_hyper_param_list,
                                        model_type=model_type, num_frames=num_frames, num_features=num_features, weights_path=weights_path)
      
      log_file_path = os.path.join(weights_path, f"model{param_idx}.log")
      save_log = CSVLogger(log_file_path)

      model = compile_model(sup_hyper_param_list=sup_hyper_param_list,lstm_units=lstm_units,
                            model_type=model_type, num_frame = num_frames, num_features = num_features)
      
      model_fit = model.fit(X_train, y_train_cat, epochs=epochs, validation_split = 0.2, batch_size=sup_hyper_param_list[3],
                            callbacks=[save_weights_json,save_log])
      
    

### Run and Fit Model with Hyperparameter Tuning

In [6]:


""" Running the desired model with several parameters using run_models():

hyper_param_list  = DO NOT CHANGE. Keep as it is in the function call.

epochs = Number of epochs you would like to try for each of the model combination

lstm_units = LSTM layer units (recommendation: do not increase it too much, start with 16,32, 64)

split_train_path = Path to train data

split_train_path = Path to test data

weights_path = Path to weights folder which all files regarding to model will be created

model_type = Which model is you want to try? 

    options for model_type are:
              LSTM
              CNN_LSTM
              BiLSTM
              CNN_BiLSTM

"""
# You CAN change param_grid depending on which parameter combinations you would like to try in the model:

param_grid = {
              'dropout_rate': [0.1],
              'batch_size': [64],
              'num_layer': [1,2],
              'optimizer': ['adam'],
              'learning_rate': [0.01]
              }

# Do not change hyper_param_list

hyper_param_list=[(num_layer, dropout_rate, optimizer, batch_size, learning_rate)
                  for num_layer in param_grid['num_layer']
                  for dropout_rate in param_grid['dropout_rate']
                  for optimizer in param_grid['optimizer']
                  for batch_size in param_grid['batch_size']
                  for learning_rate in param_grid['learning_rate']
                  ]


""" Running the desired model with several parameters using run_models():

hyper_param_list  = DO NOT CHANGE. Keep as it is in the function call.

epochs = Number of epochs you would like to try for each of the model combination

lstm_units = LSTM layer units (recommendation: do not increase it too much, start with 16,32, 64)

split_train_path = Path to train data

split_train_path = Path to test data

weights_path = Path to weights folder which all files regarding to model will be created

model_type = Which model is you want to try? 

    options for model_type are:
              LSTM
              CNN_LSTM
              BiLSTM
              CNN_BiLSTM

"""


split_train_path = r"C:\Users\Ivan Lopez\Desktop\GDP\split_seqs\run1_a+p_dist_1_0.5\train"
split_test_path = r"C:\Users\Ivan Lopez\Desktop\GDP\split_seqs\run1_a+p_dist_1_0.5\test"
weights_path = r"C:\Users\Ivan Lopez\Desktop\GDP\weights"

# --- Irem - Colab Version  ---
#split_train_path="/content/drive/MyDrive/GDP/train"
#split_test_path="/content/drive/MyDrive/GDP/test"
#weights_path="/content/drive/MyDrive/GDP/weights"

# Run a desired model using the belove line:

run_models(hyper_param_list=hyper_param_list, epochs=2, lstm_units=16,
            split_train_path=split_train_path,split_test_path=split_test_path,
            weights_path=weights_path, model_type="LSTM")

Number of configurations to run:  2
(1, 0.1, 'adam', 64, 0.01)
Epoch 1/2
53/53 [==============================] - 11s 46ms/step - loss: 0.8020 - accuracy: 0.6730 - val_loss: 0.7338 - val_accuracy: 0.7180
Epoch 2/2
53/53 [==============================] - 1s 26ms/step - loss: 0.7639 - accuracy: 0.6869 - val_loss: 0.7123 - val_accuracy: 0.7085
(2, 0.1, 'adam', 64, 0.01)
Epoch 1/2
53/53 [==============================] - 8s 49ms/step - loss: 0.8063 - accuracy: 0.6611 - val_loss: 0.7677 - val_accuracy: 0.6896
Epoch 2/2
53/53 [==============================] - 1s 13ms/step - loss: 0.7533 - accuracy: 0.6851 - val_loss: 0.7266 - val_accuracy: 0.7050


### Get the best model path (highest test accuracy) among all best combinations

In [ ]:

WEIGHTS_DIRECTORY_PATH = 'weights/'

# Attribute based on which best model weights are to be selected
SELECTION_KEY = "best_test_acc"

# Min or Max value of SELECTION_KEY to decide the best model
SELECT_MAX = True

def read_all_json_files(weights_directory):
    weights_path = weights_directory
    json_files = glob(os.path.join(weights_path,"**/*.json"), recursive=True)
    data = []
    for file in json_files:
        with open(file, 'r') as f:
            json_data = json.load(f)
            json_data['file_path'] = os.path.abspath(file)
            data.append(json_data)
    df = pd.DataFrame(data)
    return df

def get_best_model_weights_path(weights_directory, selection_criteria, select_max = True):
    df = read_all_json_files(weights_directory)
    if select_max:
        best = df.loc[df[selection_criteria].idxmax()]
    else:
        best = df.loc[df[selection_criteria].idxmin()]
    best_wt = os.path.splitext(best['file_path'])[0] + '.h5'
    print('Best model path among all combinations is:')
    return best_wt

# Run belove line to get the path of the best model (highest test accuracy) among all of the model combinations in "weights" folder:

best_model_wt_path = get_best_model_weights_path(WEIGHTS_DIRECTORY_PATH, SELECTION_KEY, SELECT_MAX)
best_model_wt_path